In [1]:
import base64
import os.path

import dotenv
from bs4 import BeautifulSoup
from google_auth_oauthlib.flow import InstalledAppFlow

from googleapiclient.discovery import build
from google_auth_oauthlib import flow
from google.auth.transport.requests import Request
import pickle
import os
import base64
import pickle
import json
import email

dotenv.load_dotenv("./.env")
API_KEY = os.getenv("API_KEY")
API_CLIENT_ID = os.getenv("API_CLIENT_ID")



In [10]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def getEmails():
    creds = None
    
    # creds
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0) 
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)
        
    service = build('gmail', 'v1', credentials=creds)
    result = service.users().messages().list(userId='me').execute()
    messages = result.get('messages')[0]
    
    print(messages)
    
    txt = service.users().messages().get(userId='me', id=messages['id']).execute()
    email = json.dumps(txt, indent=2)
    print(email)
    return txt
    
    '''
    for msg in messages:
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        #print(txt)
        
        # Cada request necesita una manera de manejar las excepciones
        payload = txt['payload']
        headers = payload['headers']
        try:
                
            for d in headers:
                
                
                if d['name'] == 'Subject':
                    subject = d['value']
                    
                if d['name'] == 'From':
                    sender = d['value']
        except:
            continue
                    
        try:
            parts = payload.get("parts")[0]
            data = parts['body']['data']
            data = data.replace("-", "+").replace("_", "/")
            decoded_data = base64.b64decode(data)
            soup = BeautifulSoup(decoded_data, 'html.parser')
            body = soup.body()
        except:
            continue
                
                
        print("subject: ", subject)
        print("from: ", sender)
        print("message: ", body)
        print("\n")
        break
        
        
                
       '''         
    

In [11]:
email_gael = getEmails()

{'id': '18f3af313d1491f9', 'threadId': '18f3af313d1491f9'}
{
  "id": "18f3af313d1491f9",
  "threadId": "18f3af313d1491f9",
  "labelIds": [
    "IMPORTANT",
    "CATEGORY_PERSONAL",
    "INBOX"
  ],
  "snippet": "",
  "payload": {
    "partId": "",
    "mimeType": "multipart/mixed",
    "filename": "",
    "headers": [
      {
        "name": "Delivered-To",
        "value": "omarafjo@gmail.com"
      },
      {
        "name": "Received",
        "value": "by 2002:a05:6f02:6599:b0:6a:551:2917 with SMTP id t25csp693643rce;        Thu, 2 May 2024 13:16:00 -0700 (PDT)"
      },
      {
        "name": "X-Received",
        "value": "by 2002:a2e:96d1:0:b0:2d8:8e1e:e15d with SMTP id d17-20020a2e96d1000000b002d88e1ee15dmr441856ljj.32.1714680959913;        Thu, 02 May 2024 13:15:59 -0700 (PDT)"
      },
      {
        "name": "ARC-Seal",
        "value": "i=1; a=rsa-sha256; t=1714680959; cv=none;        d=google.com; s=arc-20160816;        b=umMFz/lXkGFzD7mypFR45DAfKzlt8NFGTl6Jbb5HkXAoeZVYYa

In [35]:
with open('token.pickle', 'rb') as token:
    creds = pickle.load(token)
service = build('gmail', 'v1', credentials=creds)
email_id = email_gael['id']


for part in email_gael['payload']['parts']:
    if part['filename'].endswith('.pdf'):
        file = service.users().messages().attachments().get(userId = 'me',
                                                     messageId = email_id,
                                                     id=part['body']['attachmentId']).execute()



{'size': 222163,
 'data': 'JVBERi0xLjQKJcfsj6IKNSAwIG9iago8PC9MZW5ndGggNiAwIFIvRmlsdGVyIC9GbGF0ZURlY29kZT4-CnN0cmVhbQp4nO19C5hdVX2vRP2MR4pVqNJeq6uAMqEzO3vttfZjUQqcSSbJQJIJmSGKserJnpPk4D7nkDNnYqB8LSpa_ESsFh_l5vqgVa-l2iuPK16RXkGe3kqpwkVqr-IDb2yxKtL6SaH9_9drr73PI5N3qIOfX876zX-v9X_v9d47iO8FnPj4P_MjbVaWb4zJtrkKpV7iJ4y8ueJ7NIlFkgBVQhMWCkLJxtUVHoQejQPCEs68WJBmxbe_s8r2yjQ86AsmBKfwZBzwyBfwQwQigrqggiT0PcYTp4JylboaeCoKmM9j5CCMeRDg4zSIQy902y9XaLmgwqdBCE-HQciEZAeYiENZTexD0amlXK-thQnfp1gLZ77wUR0x55RLZQRQv5cUqinWa2sJE58yVAQLBPUDlC0JQ5ANawHu0BJ5LeV6VTU7CA9AFYxJm1EQhNLEo4RFXHhMGTEhK9uV8yqBx0IkRDMmQRBEEen9gU1zEXsB6C4MIzAbCuCDMXiOqKbDWHhoURb53Asp0FkkYbEXRobFCosiL_aB9SCmnmBAyeLYYyyySJYjjFMPlDEcCTmYn2C9aE2GP3zAQ5DRp7acVigwkqAaNZL1Q5wSpYEXJ8K2kJcVD1CjQSznNGAep0k_xHI-FEk8kRAaCC-JYmw5BLciOZ-qnFZy6RSS9UMiiN8wf0ppntlWnLKWp2wbtNmrSasicwHpbDtg4z2DTXWUGWYrBPGOHhma_eQ0CI0TSA57QaIQ8iKRuqUh85JcRFV0JFRAthDAGFwB1h90a3lZ8ZP286oeDVjESmH11g-BrIaa1ZqnEZeat5yqsiudQhzPG4YYv9CItYtu1ylrCXu9rxRqB2raRUMeKUOq0ITXtxeiseA1Kc0XicgLA

In [37]:
data = file['data'].replace("-", "+").replace("_", "/")
decoded_data = base64.b64decode(data)

In [39]:
with open('filename.pdf', 'wb') as filename:
    
    filename.write(decoded_data)